#  **The Convex Hull problem**

In [1]:
from __future__ import division, print_function

%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd

import collections
import random
import math
import time

Define funtion

In [2]:
# Distance-determined function
from cmath import sqrt

def distance(A, B):
    dis = sqrt((B.x - A.x)**2 + (A.y - B.y)**2) 
    return dis

In [3]:
# Turn-determined function
def turn(A, B, C):
    diff = (B.x - A.x) * (C.y - B.y)  -  (B.y - A.y) * (C.x - B.x) 
    return ('right' if diff < 0 else
            'left'  if diff > 0 else
            'straight')

Point of data


In [4]:
Point = collections.namedtuple('Point', 'x, y')
def Points(data,eliminate_rate):
    points = set()
    top = data.quantile(1-eliminate_rate)
    bottom = data.quantile(eliminate_rate)
    for i in range(len(data)):
        if ((data.loc[i].at['X']<=top['X'])&(data.loc[i].at['X']>=bottom['X'])
        &(data.loc[i].at['Y']<=top['Y'])&(data.loc[i].at['Y']>=bottom['Y'])):
            points.add(Point(data.loc[i].at['X'],data.loc[i].at['Y']))
    return points

Plot function:

In [5]:
def plot_points(points, style='r.',labels=False, closed=False):
    if labels:
        for (i, (x, y)) in enumerate(points):
            plt.text(x, y, ' '+str(i))
    if closed:
        points = points + [points[0]]
    plt.plot([p.x for p in points], [p.y for p in points], style, linewidth = 2.5)
    #plt.show()
    if closed==False: 
        print("Size of point: ", len(points))
    else:
        print("Size of hull: ", len(points)-1)

    plt.axis('scaled'); plt.axis('off')

In [6]:
def plot_partial_hull(points, hull_indexes=()):
    "Plot the points, labeled, with a blue line for the points named by indexes."
    plot_points(points, labels=True)
    plot_points([points[i] for i in hull_indexes], 'bs-')

In [7]:
def plot_convex_hull(points):
    "Show a plot."
    plot_points(points, 'bs-', closed=True)

Algorithm: 

<img src="angle.png">

In [8]:
def len_hull(hull_indexes=()):
    len = 0
    pre = Point(0,0)
    for i in hull_indexes:
        len += distance(pre,i)
        pre = i
    return len - distance(Point(0,0),hull_indexes[1])

In [9]:
def convex_hull(points):
    start_time = time.time()
    if len(points) <= 3:
        return points
    # Find the two half-hulls and append them, but don't repeat first and last points
    upper = half_hull(sorted(points))
    lower = half_hull(reversed(sorted(points)))
    length = len_hull(upper) +len_hull(lower)
    print(length)
    end_time = time.time()
    print(end_time-start_time)
    return upper + lower[1:-1]


def half_hull(sorted_points):
    "Return the half-hull from following points in sorted order."
    # Add each point C in order; remove previous point B if A->B-C is not a left turn.
    hull = []
    for C in sorted_points:
        # if A->B->C is not a left turn ...
        while len(hull) >= 2 and turn(hull[-2], hull[-1], C) != 'left':
            hull.pop() # ... then remove B from hull.
        hull.append(C)
    return hull